In [1]:
from train import train
from joblib import Parallel, delayed
from itertools import product
from tqdm.notebook import tqdm
from game_configs import *
import warnings
warnings.filterwarnings("ignore")

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
base_config = {
    "device": "cpu",
    "transitions": 100_000,
    "hidden_size": 64,
    "buffer_size": 20_000,
    "batch_size": 512,
    "actor_lr": 1e-3,
    "critic_lr": 1e-3,
    "gamma": 0.998,
    "tau": 5e-3,
    "sigma_max": 0.2,
    "sigma_min": 0,
    "render": False,
    "seed": 42,
    "return_agents": False
}

In [3]:
dtrain = delayed(train)

In [4]:
log = train(**base_config)

Filling up buffer...


  0%|          | 1/100000 [00:00<3:27:44,  8.02it/s]

Finished. Now training...


  0%|          | 63/100000 [00:05<2:29:07, 11.17it/s]

In [1]:
log.plot("step", "predator_mean", "predator_std", title="Predator")
log.plot("step", "prey_mean", "prey_std", title="Prey")

NameError: name 'log' is not defined

In [ ]:
configs = []

params = product(
    (1_000,),                 # transitions
    (10_000, 50_000,),  # buffer_size
    (0.999, 0.99),              # gamma
    (1e-3,),               # tau
    (1e-4, 1e-3),               # actor lr
    (2e-4, 2e-3),               # critic lr
    (2, 1, 0.5),                # sigma
    (256, 512, 1024),  # batch size
)

for transitions, buffer_size, gamma, tau, actor_lr, critic_lr, sigma_max, batch_size in params:
    cfg = base_config.copy()
    cfg["transitions"] = transitions
    cfg["buffer_size"] = buffer_size
    cfg["batch_size"] = batch_size
    cfg["actor_lr"] = actor_lr
    cfg["critic_lr"] = critic_lr
    cfg["gamma"] = gamma
    cfg["tau"] = tau
    cfg["sigma_max"] = sigma_max
    
    configs.append(cfg)

print(len(configs))

In [ ]:
logs = Parallel(n_jobs=8)(dtrain(**cfg) for cfg in tqdm(configs))

In [ ]:
for log in logs:
    print(log.params)
    log.plot("step", "predator_mean", "predator_std", title="Predator")
    log.plot("step", "prey_mean", "prey_std", title="Prey")